# T05 : Feature Engineering (Version 2)

## 1. Contexte et Objectifs

Ce notebook implémente la création des variables explicatives (features) qui alimenteront les modèles de Machine Learning et de Reinforcement Learning.
Conformément au cahier des charges (README), nous nous concentrons sur deux blocs de features calculées exclusivement sur le passé :

1.  **Bloc Court Terme** : Dynamique immédiate du prix (Returns, RSI, Ranges).
2.  **Bloc Contexte & Régime** : Tendances de fond (EMA 200, ADX) et volatilité (ATR).

Les données sources sont les bougies M15 nettoyées (issues de T03).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configuration "Cocooning Beige"
sns.set_theme(style="whitegrid")
plt.rcParams.update({
    "figure.facecolor": "#FAF0E6",
    "axes.facecolor": "#F5F5DC",
    "grid.color": "#E0D0C0",
    "text.color": "#5D4037",
    "axes.labelcolor": "#5D4037",
    "xtick.color": "#5D4037",
    "ytick.color": "#5D4037",
    "axes.prop_cycle": plt.cycler(color=['#8D6E63', '#A1887F', '#D7CCC8'])
})

DATA_DIR = "data/m15/clean"
FEATURES_DIR = "data/features"
os.makedirs(FEATURES_DIR, exist_ok=True)

FILES = {
    "2022": "GBPUSD_M15_2022_clean.csv",
    "2023": "GBPUSD_M15_2023_clean.csv",
    "2024": "GBPUSD_M15_2024_clean.csv"
}

## 2. Bibliothèque d'Indicateurs Techniques

Nous implémentons ici les fonctions de calcul vectorisé pour chaque indicateur, en utilisant uniquement `pandas` et `numpy` pour garantir l'indépendance vis-à-vis des librairies tierces.

In [ ]:
def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).ewm(alpha=1/period, adjust=False).mean()
    loss = (-delta.where(delta < 0, 0)).ewm(alpha=1/period, adjust=False).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_atr(df, period=14):
    high = df['high_15m']
    low = df['low_15m']
    close = df['close_15m']
    prev_close = close.shift(1)
    
    tr1 = high - low
    tr2 = (high - prev_close).abs()
    tr3 = (low - prev_close).abs()
    
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.ewm(alpha=1/period, adjust=False).mean()
    return atr

def calculate_adx(df, period=14):
    high = df['high_15m']
    low = df['low_15m']
    close = df['close_15m']
    
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    atr = calculate_atr(df, period)
    
    plus_di = 100 * (plus_dm.ewm(alpha=1/period).mean() / atr)
    minus_di = 100 * (minus_dm.abs().ewm(alpha=1/period).mean() / atr)
    
    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    adx = dx.ewm(alpha=1/period).mean()
    return adx

def calculate_macd(series, fast=12, slow=26, signal=9):
    ema_fast = series.ewm(span=fast, adjust=False).mean()
    ema_slow = series.ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    return macd_line, signal_line

## 3. Création du Feature Pack

Cette fonction principale applique l'ensemble des transformations définies dans le README à un DataFrame donné.

In [ ]:
def generate_features(df):
    df_feat = df.copy()
    close = df_feat['close_15m']
    
    # --- 6.1 Bloc Court Terme ---
    # Rendements
    df_feat['return_1'] = close.pct_change(1)
    df_feat['return_4'] = close.pct_change(4)
    
    # Moyennes Mobiles Exponentielles
    df_feat['ema_20'] = close.ewm(span=20, adjust=False).mean()
    df_feat['ema_50'] = close.ewm(span=50, adjust=False).mean()
    df_feat['ema_diff'] = df_feat['ema_20'] - df_feat['ema_50']
    
    # Oscillateurs
    df_feat['rsi_14'] = calculate_rsi(close, 14)
    
    # Volatilité locale
    df_feat['rolling_std_20'] = close.rolling(window=20).std()
    
    # Caractéristiques de bougie
    df_feat['range_15m'] = df_feat['high_15m'] - df_feat['low_15m']
    df_feat['body'] = (df_feat['close_15m'] - df_feat['open_15m']).abs()
    df_feat['upper_wick'] = df_feat['high_15m'] - df_feat[['open_15m', 'close_15m']].max(axis=1)
    df_feat['lower_wick'] = df_feat[['open_15m', 'close_15m']].min(axis=1) - df_feat['low_15m']
    
    # --- 6.2 Bloc Contexte & Régime ---
    # Tendance Long Terme
    df_feat['ema_200'] = close.ewm(span=200, adjust=False).mean()
    df_feat['distance_to_ema200'] = close - df_feat['ema_200']
    # Pente EMA 50 (approchée par la variation sur 3 périodes)
    df_feat['slope_ema50'] = df_feat['ema_50'].diff(3)
    
    # Régime de Volatilité
    df_feat['atr_14'] = calculate_atr(df_feat, 14)
    df_feat['rolling_std_100'] = close.rolling(window=100).std()
    # Ratio volatilité court terme / long terme
    df_feat['volatility_ratio'] = df_feat['rolling_std_20'] / df_feat['rolling_std_100']
    
    # Force Directionnelle
    df_feat['adx_14'] = calculate_adx(df_feat, 14)
    df_feat['macd'], df_feat['macd_signal'] = calculate_macd(close)
    
    # Nettoyage des NaN générés par les fenêtres glissantes (ex: EMA 200)
    # On supprime le début de l'historique (warm-up)
    df_feat.dropna(inplace=True)
    
    return df_feat

## 4. Application et Sauvegarde

Nous appliquons cette transformation à chaque année séparément, en veillant à la cohérence. Notez que pour une application en production ou le backtesting 'walk-forward', le calcul des indicateurs devrait idéalement se faire sur un flux continu pour éviter les effets de bord au 1er janvier. Ici, nous traitons chaque année comme un bloc, ce qui implique une perte des 200 premières bougies pour le 'warm-up'.

In [ ]:
feature_sets = {}

for year, filename in FILES.items():
    input_path = os.path.join(DATA_DIR, filename)
    if not os.path.exists(input_path):
        print(f"Fichier {input_path} introuvable, passé.")
        continue
        
    print(f"Traitement {year}...")
    df_raw = pd.read_csv(input_path, parse_dates=['timestamp'], index_col='timestamp')
    
    # Génération features
    df_features = generate_features(df_raw)
    feature_sets[year] = df_features
    
    # Sauvegarde
    output_path = os.path.join(FEATURES_DIR, f"GBPUSD_M15_{year}_features.csv")
    df_features.to_csv(output_path)
    print(f" -> Features calculées : {df_features.shape[1]} colonnes. Sauvegardé dans {output_path}")

## 5. Visualisation des Régimes

Vérifions visuellement la pertinence des indicateurs de régime (ADX et Volatilité).

In [ ]:
if '2023' in feature_sets:
    sample = feature_sets['2023'].iloc[1000:1500] # Un échantillon de 500 bougies
    
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 12), sharex=True)
    
    # Prix + EMAs
    ax1.plot(sample.index, sample['close_15m'], label='Close', color='#5D4037', alpha=0.8)
    ax1.plot(sample.index, sample['ema_50'], label='EMA 50', color='#D7CCC8', linestyle='--')
    ax1.plot(sample.index, sample['ema_200'], label='EMA 200', color='#8D6E63', linewidth=1.5)
    ax1.set_title("Prix et Tendances (EMA)", fontweight='bold')
    ax1.legend()
    
    # ADX (Force de la tendance)
    ax2.plot(sample.index, sample['adx_14'], label='ADX 14', color='#A1887F')
    ax2.axhline(25, color='gray', linestyle=':', alpha=0.5, label='Seuil Trend (25)')
    ax2.set_title("Force de Tendance (ADX)")
    ax2.set_ylabel("0-100")
    ax2.legend()
    
    # Volatilité (ATR)
    ax3.plot(sample.index, sample['atr_14'], label='ATR 14', color='#BCAAA4')
    ax3.set_title("Volatilité (ATR)")
    ax3.set_xlabel("Date")

    plt.show()

## 6. Conclusion T05

Le Feature Engineering est terminé. Nous disposons désormais de datasets enrichis contenant :
- L'information OHLCV de base.
- Les indicateurs de dynamique (RSI, Returns).
- Les indicateurs de contexte (Trend, Volatilité).

Ces fichiers `_features.csv` seront l'entrée directe pour l'entraînement des modèles (T07) et l'environnement de Reinforcement Learning (T08).